In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature_pressure import*
co2_param_file = os.path.join(root_path,'iltransr/pre-trained params/co2_best.params')

[03:45:07] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[03:45:08] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0
batch_size = 64

In [3]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()
print(net)

ILNet(
  (IL_src_embed): HybridSequential(
    (0): Embedding(72 -> 128, float32)
    (1): Dropout(p = 0.0, axes=())
  )
  (IL_encoder): TransformerEncoder(
    (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
    (transformer_cells): HybridSequential(
      (0): TransformerEncoderCell(
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0, axes=())
          )
          (proj_query): Dense(128 -> 128, linear)
          (proj_key): Dense(128 -> 128, linear)
          (proj_value): Dense(128 -> 128, linear)
        )
        (proj): Dense(128 -> 128, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(128 -> 1024, linear)
          (activation): Activation(relu)
          (ffn_2): Dense(1024 -> 128, linear)
          (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
        )
        (layer_norm): LayerNorm(eps=1e-

In [4]:
net.load_parameters(co2_param_file,ctx=ctx)
net.initialize(init=mx.init.Xavier(magnitude=magnitude), ctx=ctx)

In [6]:
import pandas as pd
co2_database = pd.read_excel('smiles.xlsx',sheet_name='To_be_calculated')

In [7]:
train_IL_smiles = co2_database['SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_T =co2_database['normalized_T_co2']
train_P = co2_database['normalized_P_co2']
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_IL_smiles,train_T,train_P))
predict_dataloader = get_predict_dataloader(train_dataset, batch_size=batch_size)

In [8]:
predicted = predict(net, predict_dataloader,ctx)

[03:45:33] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable
[03:45:34] ../src/operator/cudnn_ops.cc:292: Auto-tuning cuDNN op, set MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable


In [9]:
df = pd.DataFrame(predicted)
df.to_csv('predicted_co2.csv')
print(df)

             0
0     0.013991
1     0.018719
2     0.017333
3     0.015623
4     0.014399
...        ...
5513  0.051523
5514  0.012942
5515  0.088111
5516  0.014189
5517  0.014892

[5518 rows x 1 columns]
